# Data cleaning for SEER 1y incidence

In [2]:
import numpy as np
import pandas as pd
import os
os.getcwd() 

'c:\\repos\\pdac-calibration\\notebooks'

In [87]:
seer_inc = pd.read_csv("../data/pdac_seer17_incidence_1y.csv")
seer_inc.iloc[:5]

,Year Dx (2000-2021 group only) for DevCan,Age recode with single ages and 85+,Combined Summary Stage (2004+),Crude Rate,Count,Population
0,2000-2021,00 years,In situ,0.000000,0,23639727
1,2000-2021,00 years,Localized,0.000000,0,23639727
2,2000-2021,00 years,Regional,0.000000,0,23639727
3,2000-2021,00 years,Distant,0.000000,0,23639727
4,2000-2021,00 years,NaN,0.000000,0,23639727


In [89]:
# Pivot the data
pivot_df = seer_inc.pivot(
    index='Age recode with single ages and 85+', 
    columns='Combined Summary Stage (2004+)', 
    values='Crude Rate'
)

pivot_df.columns = [f"{col} Rate" for col in pivot_df.columns]
pivot_df.reset_index(inplace=True)
pivot_df = pivot_df[pivot_df['Age recode with single ages and 85+'] != 'Unknown']
pivot_df['Age'] = pivot_df['Age recode with single ages and 85+'].str[:2].astype(int)
pivot_df['Localized Rate'] = pivot_df['Localized Rate'].astype(float)
pivot_df['Regional Rate'] = pivot_df['Regional Rate'].astype(float)
pivot_df['Distant Rate'] = pivot_df['Distant Rate'].astype(float)
pivot_df['Unknown/unstaged Rate'] = pivot_df['Unknown/unstaged Rate'].astype(float)

In [90]:
pivot_df.drop(['nan Rate', 'In situ Rate', 'Blank(s) Rate', 'Age recode with single ages and 85+'], axis=1, inplace=True)

In [91]:
pivot_df.head()

,Distant Rate,Localized Rate,Regional Rate,Unknown/unstaged Rate,Age
0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,2
3,0.0,0.0,0.0,0.0,3
4,0.0,0.0,0.0,0.0,4


Redistribute unstaged cancer incidence over local, regional, and distant

In [92]:
pivot_df['Distant Rate'].iloc[4]

0.0

In [93]:
pivot_df['LRD Rate'] = pivot_df['Localized Rate'] + pivot_df['Regional Rate'] + pivot_df['Distant Rate']
pivot_df['LRDU Rate'] = pivot_df['Localized Rate'] + pivot_df['Regional Rate'] + pivot_df['Distant Rate'] + pivot_df['Unknown/unstaged Rate']
pivot_df['pLocal'] = pivot_df['Localized Rate'] / pivot_df['LRD Rate']
pivot_df['pRegional'] = pivot_df['Regional Rate'] / pivot_df['LRD Rate']
pivot_df['pDistant'] = pivot_df['Distant Rate'] / pivot_df['LRD Rate']
pivot_df['LocalU'] = pivot_df['Localized Rate'] + pivot_df['Unknown/unstaged Rate'] * pivot_df['pLocal']
pivot_df['RegionalU'] = pivot_df['Regional Rate'] + pivot_df['Unknown/unstaged Rate'] * pivot_df['pRegional']
pivot_df['DistantU'] = pivot_df['Distant Rate'] + pivot_df['Unknown/unstaged Rate'] * pivot_df['pDistant']

In [94]:
pivot_df = pivot_df[pivot_df['Age']>=20]
pivot_df = pivot_df[pivot_df['Age']<85]

In [11]:
pivot_df.to_csv("../data/seer_incidence_1y.csv")